<a href="https://colab.research.google.com/github/MuhammadShavaiz/AI_learning/blob/main/bigram_LLM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torch.nn as nn

In [2]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()
print(len(text))

252022


In [3]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
92


In [4]:
char_to_int = {ch:i for i ,ch in enumerate(chars)}
int_to_char = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [char_to_int[c] for c in x]
decode = lambda y: ''.join([int_to_char[i] for i in y])
encoded_shavaiz = encode('shavaiz')
decoded_shavaiz = decode(encoded_shavaiz)
print(encoded_shavaiz, decoded_shavaiz)

[76, 65, 58, 79, 58, 66, 83] shavaiz


In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [6]:
print(decode(data[:100].numpy()))

﻿The Project Gutenberg eBook of Dorothy and the Wizard in Oz
    
This ebook is for the use of anyon


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
block_size = 8
batch_size = 4

In [13]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]
def get_batch(split):
  data = train_data if split == 'train' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(ix)
  x = torch.stack([data[i:i+block_size]for i in ix])
  y = torch.stack([data[i+1:i+block_size+1]for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y
x,y = get_batch('train')
print(f'x:{x}')
print(f'y:{y}')

tensor([ 57171,  53418,   2802, 162407])
x:tensor([[58, 75, 72, 78, 71, 61,  1, 77],
        [82,  1, 67, 78, 61, 64, 70, 62],
        [ 1, 37,  1, 61, 72,  1, 59, 78],
        [75, 76, 12,  1, 77, 72, 72, 14]])
y:tensor([[75, 72, 78, 71, 61,  1, 77, 65],
        [ 1, 67, 78, 61, 64, 70, 62, 71],
        [37,  1, 61, 72,  1, 59, 78, 77],
        [76, 12,  1, 77, 72, 72, 14,  3]])


In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f'when context is {context}, target is {target}')

when context is tensor([91]), target is 48
when context is tensor([91, 48]), target is 65
when context is tensor([91, 48, 65]), target is 62
when context is tensor([91, 48, 65, 62]), target is 1
when context is tensor([91, 48, 65, 62,  1]), target is 44
when context is tensor([91, 48, 65, 62,  1, 44]), target is 75
when context is tensor([91, 48, 65, 62,  1, 44, 75]), target is 72
when context is tensor([91, 48, 65, 62,  1, 44, 75, 72]), target is 67


example of extracing a word of four letter from vector space

In [10]:
vocab_size = 10000
embedding_dim = 100
embedding = nn.Embedding(vocab_size, embedding_dim)
input_indices = torch.LongTensor([1,5,3,2])
embedded_output = embedding(input_indices)
print(embedded_output.shape)

torch.Size([4, 100])


In [14]:
class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
  def forward(self,index,targets):
    logits = self.token_embedding_table(index)
    return logits